In [55]:
import requests
import pandas as pd

In [56]:
# URL de l'API pour le classement Ligue 1 2024–2025
url = "https://www.thesportsdb.com/api/v1/json/3/lookuptable.php?l=4334&s=2024-2025"

# Requête à l'API
response = requests.get(url)
r = response.json()

# Vérification de la clé 'table'
if 'table' not in r:
    print("❌ Données de classement introuvables.")
else:
    table = r['table']

    # Construction du DataFrame
    data = pd.DataFrame([{
        "Pos": team["intRank"],
        "Team": team["strTeam"],
        "M": team["intPlayed"],
        "W": team["intWin"],
        "D": team["intDraw"],
        "L": team["intLoss"],
        "G": team["intGoalsFor"],
        "GA": team["intGoalsAgainst"],
        "Diff": team["intGoalDifference"],
        "PTS": team["intPoints"]
    } for team in table])

    # Tri par position
    data = data.sort_values(by="PTS").reset_index(drop=True)

    # Affichage
    print("🏆 Classement Ligue 1 – 2024/2025 :")
    display(data)

🏆 Classement Ligue 1 – 2024/2025 :


,Pos,Team,M,W,D,L,G,GA,Diff,PTS
0,18,Montpellier,33,4,4,25,23,76,-53,16
1,17,St Etienne,33,8,6,19,37,74,-37,30
2,16,Le Havre,33,9,4,20,37,69,-32,31
3,15,Nantes,33,7,12,14,36,52,-16,33
4,14,Stade de Reims,33,8,9,16,32,45,-13,33
5,13,Angers,33,10,6,17,32,51,-19,36
6,12,Toulouse,33,10,9,14,41,41,0,39
7,11,Rennes,33,13,2,18,49,46,3,41
8,10,Auxerre,33,11,9,13,47,48,-1,42
9,9,Lens,33,14,7,12,38,39,-1,49


In [57]:
# Conversion sécurisée de toutes les colonnes en int à part Team
for col in data.columns:
    if col != "Team":
        data[col] = pd.to_numeric(data[col], errors='coerce').fillna(0).astype(int)

print(data.columns)
print(len(data.columns))

Index(['Pos', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'Diff', 'PTS'], dtype='object')
10


In [58]:
# Rename columns for easier handling
data.columns = ['Pos', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'Diff', 'PTS']
data.head()

,Pos,Team,M,W,D,L,G,GA,Diff,PTS
0,18,Montpellier,33,4,4,25,23,76,-53,16
1,17,St Etienne,33,8,6,19,37,74,-37,30
2,16,Le Havre,33,9,4,20,37,69,-32,31
3,15,Nantes,33,7,12,14,36,52,-16,33
4,14,Stade de Reims,33,8,9,16,32,45,-13,33


In [59]:
# League averages
league_avg_goals_for = data['G'].sum() / data['M'].sum()
league_avg_goals_against = data['GA'].sum() / data['M'].sum()

print(f"League Avg Goals Scored per Match: {league_avg_goals_for:.2f}")

League Avg Goals Scored per Match: 1.47


In [60]:
# Calculate team attack and defense strength
data['Attack_Strength'] = (data['G'] / data['M']) / league_avg_goals_for
data['Defense_Weakness'] = (data['GA'] / data['M']) / league_avg_goals_for

data[['Team', 'Attack_Strength', 'Defense_Weakness']].head()

,Team,Attack_Strength,Defense_Weakness
0,Montpellier,0.474227,1.567010
1,St Etienne,0.762887,1.525773
2,Le Havre,0.762887,1.422680
3,Nantes,0.742268,1.072165
4,Stade de Reims,0.659794,0.927835


In [61]:
def predict_match(team1, team2, data, max_goals=5, print_output=True):
    import numpy as np
    from scipy.stats import poisson

    if team1 not in data['Team'].values:
        raise ValueError(f"Équipe inconnue : {team1}")
    if team2 not in data['Team'].values:
        raise ValueError(f"Équipe inconnue : {team2}")

    # League average goals (you can also pass this in if you want to optimize)
    league_avg_goals = data['G'].sum() / data['M'].sum()

    # Get team stats
    t1 = data[data['Team'] == team1].iloc[0]
    t2 = data[data['Team'] == team2].iloc[0]

    # Expected goals using Poisson assumption
    exp_g1 = t1['Attack_Strength'] * t2['Defense_Weakness'] * league_avg_goals
    exp_g2 = t2['Attack_Strength'] * t1['Defense_Weakness'] * league_avg_goals

    # Poisson probability matrix
    prob_matrix = np.outer(
        [poisson.pmf(i, exp_g1) for i in range(max_goals + 1)],
        [poisson.pmf(j, exp_g2) for j in range(max_goals + 1)]
    )

    # Match outcome probabilities
    home_win_prob = np.tril(prob_matrix, -1).sum()
    draw_prob = np.trace(prob_matrix)
    away_win_prob = np.triu(prob_matrix, 1).sum()

    # Decimal odds (implied, without margin)
    home_odds = round(1 / home_win_prob, 2)
    draw_odds = round(1 / draw_prob, 2)
    away_odds = round(1 / away_win_prob, 2)

    if print_output:
        print(f"\n📊 Expected Goals:")
        print(f"{team1}: {exp_g1:.2f}, {team2}: {exp_g2:.2f}")

        print(f"\n📈 Match Outcome Probabilities:")
        print(f"{team1} Win: {home_win_prob:.2%}")
        print(f"Draw: {draw_prob:.2%}")
        print(f"{team2} Win: {away_win_prob:.2%}")

        print(f"\n🎯 Implied Decimal Odds:")
        print(f"{team1} Win: {home_odds}")
        print(f"Draw: {draw_odds}")
        print(f"{team2} Win: {away_odds}")

    return {
                'expected_goals': {team1: exp_g1, team2: exp_g2},
        'probabilities': {
            f'{team1}_win': home_win_prob,
            'draw': draw_prob,
            f'{team2}_win': away_win_prob,
        },
        'decimal_odds': {
            f'{team1}_win': home_odds,
            'draw': draw_odds,
            f'{team2}_win': away_odds,
        },
        'probability_matrix': prob_matrix
    }

In [62]:
predict_match('Paris SG', 'Auxerre', data)


📊 Expected Goals:
Paris SG: 2.67, Auxerre: 1.00

📈 Match Outcome Probabilities:
Paris SG Win: 67.38%
Draw: 15.48%
Auxerre Win: 11.65%

🎯 Implied Decimal Odds:
Paris SG Win: 1.48
Draw: 6.46
Auxerre Win: 8.59


{'expected_goals': {'Paris SG': np.float64(2.669165885660731),
  'Auxerre': np.float64(0.9984379881287097)},
 'probabilities': {'Paris SG_win': np.float64(0.6738384787980847),
  'draw': np.float64(0.15483989179481913),
  'Auxerre_win': np.float64(0.11647786875229546)},
 'decimal_odds': {'Paris SG_win': np.float64(1.48),
  'draw': np.float64(6.46),
  'Auxerre_win': np.float64(8.59)},
 'probability_matrix': array([[0.02553759, 0.0254977 , 0.01272894, 0.00423635, 0.00105743,
         0.00021116],
        [0.06816406, 0.06805759, 0.03397564, 0.01130752, 0.00282247,
         0.00056361],
        [0.09097059, 0.09082849, 0.04534331, 0.01509083, 0.00376681,
         0.00075219],
        [0.08093853, 0.0808121 , 0.04034294, 0.01342664, 0.00335142,
         0.00066924],
        [0.05400959, 0.05392523, 0.0269205 , 0.00895948, 0.00223637,
         0.00044658],
        [0.02883211, 0.02878708, 0.01437106, 0.00478287, 0.00119385,
         0.0002384 ]])}